In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd
from sklearn import dummy, metrics, linear_model, model_selection, neighbors
from sklearn.model_selection import train_test_split

# Introdução ao Aprendizado de Máquina

## Trabalhando com dados

### Carregando os datasets

Nós iremos ver duas vertentes do mesmo dataset, com propósitos diferentes. `Weather` é um dataset educacional (composto por dados fictícios) amplamente usado para ensinar conceitos de aprendizado de máquina. Ele contém 14 instâncias, 4 features e é considerado um dataset de classificação binária.

In [3]:
weather_nominal = pd.read_csv('./weather.nominal.csv')
weather_nominal

weather_numeric = pd.read_csv('./weather.csv')
weather_numeric

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


Aqui vamos conferir os possíveis valores únicos em colunas categóricas:

In [4]:
print('Outlook:', weather_nominal.outlook.unique())
print('Temperature:', weather_nominal.temperature.unique())
print('Humidity:', weather_nominal.humidity.unique())
print('Windy:', weather_nominal.windy.unique())
print('Play:', weather_nominal.play.unique())

Outlook: ['sunny' 'overcast' 'rainy']
Temperature: ['hot' 'mild' 'cool']
Humidity: ['high' 'normal']
Windy: [False  True]
Play: ['no' 'yes']


Mostrando dataset com `head` (5 primeiros valores) e `tail` (5 últimos valores), bom para datasets extensos.

In [5]:
weather_nominal.head()
weather_numeric.tail()

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


,outlook,temperature,humidity,windy,play
9,rainy,75,80,False,yes
10,sunny,75,70,True,yes
11,overcast,72,90,True,yes
12,overcast,81,75,False,yes
13,rainy,71,91,True,no


### Transformando os dados categóricos em numéricos

Para usar os datasets, nós devemos transformar os dados categóricos em valores numéricos, já que tanto nosso método de classificação quanto o de regressão trabalham apenas com números. Iremos usar uma técnica de encoding básica, mapeando os valores categóricos para números.

Uma forma mais "profissional" de fazer a transformação de dados é o `One-hot encoding`, mas não será tratado aqui.

In [6]:
# Mudando o tipo das colunas categóricas
weather_numeric['outlook'] = weather_numeric['outlook'].astype('category')
weather_numeric['windy'] = weather_numeric['windy'].astype('category')
weather_numeric['play'] = weather_numeric['play'].astype('category')

# Aplicando enconding às colunas selecionadas
cat_columns = weather_numeric.select_dtypes(['category']).columns
weather_numeric[cat_columns] = weather_numeric[cat_columns].apply(lambda x: x.cat.codes)
weather_numeric.head()

,outlook,temperature,humidity,windy,play
0,2,85,85,0,0
1,2,80,90,1,0
2,0,83,86,0,1
3,1,70,96,0,1
4,1,68,80,0,1


### Dividindo datasets em treino e teste

In [7]:
X_nominal_train, X_nominal_test, y_nominal_train, y_nominal_test = train_test_split(
    weather_nominal.iloc[:, :-1], 
    weather_nominal.iloc[:, -1], 
    test_size=0.3
)

X_nominal_train
X_nominal_test
y_nominal_train
y_nominal_test

,outlook,temperature,humidity,windy
9,rainy,mild,normal,False
3,rainy,mild,high,False
0,sunny,hot,high,False
7,sunny,mild,high,False
2,overcast,hot,high,False
4,rainy,cool,normal,False
1,sunny,hot,high,True
11,overcast,mild,high,True
6,overcast,cool,normal,True


,outlook,temperature,humidity,windy
5,rainy,cool,normal,True
12,overcast,hot,normal,False
8,sunny,cool,normal,False
13,rainy,mild,high,True
10,sunny,mild,normal,True


9     yes
3     yes
0      no
7      no
2     yes
4     yes
1      no
11    yes
6     yes
Name: play, dtype: object

5      no
12    yes
8     yes
13     no
10    yes
Name: play, dtype: object

In [8]:
X_numeric_train, X_numeric_test, y_numeric_train, y_numeric_test = train_test_split(
    weather_numeric.iloc[:, :-1], 
    weather_numeric.iloc[:, -1], 
    test_size=0.3
)

X_numeric_train
X_numeric_test
y_numeric_train
y_numeric_test

,outlook,temperature,humidity,windy
5,1,65,70,1
6,0,64,65,1
13,1,71,91,1
1,2,80,90,1
9,1,75,80,0
3,1,70,96,0
2,0,83,86,0
0,2,85,85,0
12,0,81,75,0


,outlook,temperature,humidity,windy
4,1,68,80,0
10,2,75,70,1
11,0,72,90,1
8,2,69,70,0
7,2,72,95,0


5     0
6     1
13    0
1     0
9     1
3     1
2     1
0     0
12    1
Name: play, dtype: int8

4     1
10    1
11    1
8     1
7     0
Name: play, dtype: int8

## Modelos

### Classificação

Vamos usar o KNN para classificar as instâncias do `Weather`. Como funciona o KNN? Imagine que suas instâncias estão distribuídas pelo espaço (então você tem a ideia de pontos e o conceito de distâncias) e que instâncias próximas são semelhantes. Logo, você pode "adivinhar" a classe de uma instância olhando para seus vizinhos.  

In [9]:
classification = neighbors.KNeighborsClassifier(n_neighbors=3)
classification.fit(X_numeric_train, y_numeric_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [10]:
class_predict = classification.predict(X_numeric_test)
class_predict

array([0, 1, 0, 1, 0], dtype=int8)

### Regressão

Vamos agora usar a Regressão Linear para fazer a regressão das instâncias do `Weather`. A ideia da regressão linear é explicar seus dados como uma função `y = a * x + b + e`, onde `a` e `b` são coeficientes e `e` é um termo de erro, ou seja, uma medida que mostra que existem outros fatores não explicados pelo modelo.

In [11]:
regression = linear_model.LinearRegression()
regression.fit(X_numeric_train, y_numeric_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [12]:
regr_predict = regression.predict(X_numeric_test)
regr_predict

array([ 0.91362021, -0.28710622,  0.65729027,  0.45110359,  0.46459723])

### Avaliação de resultados

Existem várias formas de medir quão "bom" é um modelo (apesar de isso ser subjetivo), as métricas são boas formas. Elas testam resultados gerados pelos modelos com valores reais, e dão uma ideia de quão preciso é o seu modelo na classificação (ou quão bem sua curva explica os dados na regressão).

In [13]:
print('KNN:')
print(metrics.classification_report(y_numeric_test, class_predict))

print('Regressão:')
print(metrics.mean_squared_error(y_numeric_test, regr_predict))

KNN:
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         1
           1       1.00      0.50      0.67         4

   micro avg       0.60      0.60      0.60         5
   macro avg       0.67      0.75      0.58         5
weighted avg       0.87      0.60      0.63         5

Regressão:
0.4597383395421244


### Cross validation

A ideia é evitar o overfitting fazendo passos intermediários de validação com uma parte do dataset de treino. Por exemplo, uma validação cruzada de 10 folds treina o modelo com 9 pedaços e valida os resultados obtidos com o pedaço restante, e repete esse processo mais 9 vezes.

In [14]:
print('KNN:')
print('validação cruzada:', model_selection.cross_validate(
    classification, 
    X_numeric_train, 
    y_numeric_train,
    cv=2
))

print('Regressão:')
print('validação cruzada:', model_selection.cross_validate(
    regression, 
    X_numeric_train, 
    y_numeric_train,
    cv=2
))

KNN:
validação cruzada: {'fit_time': array([0.015661  , 0.00339317]), 'score_time': array([0.02187538, 0.0083282 ]), 'test_score': array([0.6 , 0.25]), 'train_score': array([0.75, 0.4 ])}
Regressão:
validação cruzada: {'fit_time': array([0.0084095 , 0.02097845]), 'score_time': array([0.00701356, 0.01187325]), 'test_score': array([-0.85396599, -8.81199539]), 'train_score': array([1., 1.])}


## Extras

### Baseline classifier

É muito difícil avaliar o resultado de um modelo (descobrir se o valor resultante das métricas é bom ou não), mas existe uma forma de descobrir qual seria o pior valor possível. A ideia é usar um classificador "dummy", isto é, uma estratégia ingênua de classificação. Neste caso apresentado abaixo, o dummy checa qual a classe dominante e classifica as instâncias de teste com aquela mesma classe.

In [15]:
classification = dummy.DummyClassifier(strategy='most_frequent')
classification.fit(X_nominal_train, y_nominal_train)

DummyClassifier(constant=None, random_state=None, strategy='most_frequent')

In [16]:
pred = classification.predict(X_nominal_test)

metrics.accuracy_score(y_nominal_test, pred)

0.6